In [1]:
from tensorflow.keras.applications.resnet import preprocess_input 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
traindf = pd.read_csv('../input/dog-breed-identification/labels.csv')

In [3]:
traindf = train_df.loc[(train_df.breed == 'beagle') | (train_df.breed == 'doberman') | (train_df.breed == 'chihuahua') | (train_df.breed == 'french_bulldog') | (train_df.breed == 'golden_retriever') | (train_df.breed == 'malamute') | (train_df.breed == 'pug') | (train_df.breed == 'saint_bernard') | (train_df.breed == 'scottish_deerhound') | (train_df.breed == 'tibetan_mastiff')].reset_index(drop=True)

In [4]:
traindf

,id,breed
0,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
1,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
2,008b1271ed1addaccf93783b39deab45,doberman
3,00a366d4b4a9bbb6c8a63126697b7656,golden_retriever
4,0100f55e4f0fe28f2c0465d3fc4b9897,golden_retriever
...,...,...
836,fe50bac6c389d137ea01c9cfc7346ca8,scottish_deerhound
837,fe624532170510bd80627c0500bafc97,tibetan_mastiff
838,feb16cf86c9dac6d476e3c372ba5c279,pug
839,febcab8eb2da444bf83336cffec7eb92,golden_retriever


In [5]:
traindf.breed.value_counts()

scottish_deerhound    126
beagle                105
pug                    94
saint_bernard          84
malamute               81
doberman               74
chihuahua              71
french_bulldog         70
tibetan_mastiff        69
golden_retriever       67
Name: breed, dtype: int64

In [6]:
train_paths = '../input/dog-breed-identification/train/' + traindf['id'] + '.jpg'

In [7]:
labels = pd.get_dummies(traindf.breed).values

In [8]:
column_names = pd.get_dummies(traindf.breed).columns.to_list()

In [9]:
train_path, valid_path, train_labels, valid_labels = train_test_split(train_paths, labels, test_size=0.3, random_state=42)

In [10]:
train_ds = tf.data.Dataset.from_tensor_slices((train_path, train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_path, valid_labels))

In [11]:
def decode_train_data(image_path, label):
    
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.image.resize(img,[312,312])
    
    return img, label

In [12]:
train_ds = train_ds.map(decode_train_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_ds = valid_ds.map(decode_train_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [13]:
def augment(img, label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    return img, label

In [14]:
def configure_for_performance(ds, batch_size = 16):
    
    ds = ds.cache('/kaggle/dump.tfcache')
    ds = ds.map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.repeat()
    ds = ds.shuffle(buffer_size=25)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

In [15]:
train_ds_batch = configure_for_performance(train_ds)
valid_ds_batch = valid_ds.batch(32)

In [16]:
base_model = tf.keras.applications.ResNet50(include_top=False,weights="imagenet")
for layers in base_model.layers:
    layers.trainable=True

94773248/94765736 [==============================] - 1s 0us/step


In [17]:
def build_model():
    inputs = tf.keras.layers.Input(shape=(312, 312, 3))
    preprocess = preprocess_input(inputs)
    outputs_resnet = base_model(preprocess)
    global_avg_pooling = tf.keras.layers.GlobalAveragePooling2D()(outputs_resnet)
    dense_1= tf.keras.layers.Dense(512, kernel_regularizer = 'l2')(global_avg_pooling)
    bn_1 = tf.keras.layers.BatchNormalization()(dense_1)
    activation = tf.keras.layers.Activation('relu')(bn_1)
    dropout = tf.keras.layers.Dropout(0.4)(activation)
    dense_2 = tf.keras.layers.Dense(10, activation='softmax')(dropout)
    
    
    model = tf.keras.Model(inputs, dense_2)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr = 3e-4),
        loss='categorical_crossentropy',
        metrics='acc'
    )
    
    return model

In [18]:
model = build_model()

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 312, 312, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 312, 312, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 312, 312, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048  

In [19]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    './model.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience=4, min_lr=1e-5, mode='min')

In [20]:
steps_per_epoch = len(train_paths) // 16
epochs = 60
history = model.fit(
                train_ds_batch, 
                validation_data = valid_ds_batch, 
                epochs = epochs,
                callbacks=[checkpoint, lr_reducer],
                steps_per_epoch = steps_per_epoch
)

Epoch 1/60
52/52 [==============================] - 23s 266ms/step - loss: 10.7523 - acc: 0.1879 - val_loss: 10.0355 - val_acc: 0.1225
Epoch 2/60
52/52 [==============================] - 11s 203ms/step - loss: 9.6879 - acc: 0.3346 - val_loss: 9.7512 - val_acc: 0.1225
Epoch 3/60
52/52 [==============================] - 10s 199ms/step - loss: 8.8106 - acc: 0.3769 - val_loss: 9.1737 - val_acc: 0.1581
Epoch 4/60
52/52 [==============================] - 10s 196ms/step - loss: 7.6858 - acc: 0.5402 - val_loss: 8.5482 - val_acc: 0.0909
Epoch 5/60
52/52 [==============================] - 10s 196ms/step - loss: 6.7078 - acc: 0.6648 - val_loss: 8.0959 - val_acc: 0.0909
Epoch 6/60
52/52 [==============================] - 10s 196ms/step - loss: 6.0582 - acc: 0.6765 - val_loss: 7.6787 - val_acc: 0.1581
Epoch 7/60
52/52 [==============================] - 10s 195ms/step - loss: 5.1407 - acc: 0.8195 - val_loss: 52.5754 - val_acc: 0.0988
Epoch 8/60
52/52 [==============================] - 10s 197ms/step